## **Federated Learning for attack classification: 3 nodes sharing weights**

IDs from this file = **ID3xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [31]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [33]:
# Preprocessing for test and training datasets (to be able to train and make predictions to evaluate)
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Creating categories dataframe
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)
        
    # Transform to One Hot Encoding the Categories - normal, dos, reconnaissance, generic, exploits, worms, fuzzers, analysis, backdoor, shellcode
    data_labels.insert(0, 'dos', data['attack_cat'].replace('dos', 1).replace(['reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(1, 'reconnaissance', data['attack_cat'].replace('reconnaissance', 1).replace([ 'dos', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(2, 'generic', data['attack_cat'].replace('generic', 1).replace(['dos', 'reconnaissance', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(3, 'exploits', data['attack_cat'].replace('exploits', 1).replace([ 'dos', 'reconnaissance', 'generic', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Generate 2 new columns to fit with training
    auxCol=data_features['sbytes']
    auxCol=0

      # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [34]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(Dense(108, input_shape=input_shape, activation='relu'))
    model.add(layers.Conv2D(filters=16, kernel_size=(1, 2), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(108, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    return model 

In [35]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, labels = [i for i in range(4)], target_names=['dos', 'reconnaissance', 'generic', 'exploits'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

### Experiments with datasets

#### Dataset 3A

In [36]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3A-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_11572/2733919938.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')


In [37]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [38]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID300.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [39]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [40]:
global_model = build_model((24,1,1))

In [41]:
# Filter test and train by not getting normal traffic: label is 1
test_basic = test_basic.loc[test_basic['label'] == 1]
test_plus = test_plus.loc[test_plus['label'] == 1]

In [42]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [43]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id300.hdf5')

Epoch 1/5
51/51 [==============================] - 3s 59ms/step - loss: 0.4610 - accuracy: 0.9267 - val_loss: 1.3888 - val_accuracy: 0.1524
Epoch 2/5
51/51 [==============================] - 3s 56ms/step - loss: 0.0271 - accuracy: 0.9994 - val_loss: 1.4406 - val_accuracy: 0.1439
Epoch 3/5
51/51 [==============================] - 3s 59ms/step - loss: 0.0092 - accuracy: 0.9999 - val_loss: 1.4983 - val_accuracy: 0.1428
Epoch 4/5
51/51 [==============================] - 3s 55ms/step - loss: 0.0048 - accuracy: 0.9999 - val_loss: 1.5822 - val_accuracy: 0.1424
Epoch 5/5
51/51 [==============================] - 3s 54ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 1.7133 - val_accuracy: 0.1420
Epoch 1/5
51/51 [==============================] - 3s 60ms/step - loss: 3.6558e-04 - accuracy: 1.0000 - val_loss: 2.3970 - val_accuracy: 0.1415
Epoch 2/5
51/51 [==============================] - 3s 56ms/step - loss: 3.4561e-05 - accuracy: 1.0000 - val_loss: 3.1075 - val_accuracy: 0.1414
Epoch 3/5
51

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 39/756 [>.............................] - ETA: 1s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 73/756 [=>............................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [44]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.7941008806228638, 0.0], [2.583343505859375, 0.0], [3.069974184036255, 0.0]]
Accuracy for iterations:  [[0.8319951891899109, 0.0], [0.8319951891899109, 0.0], [0.8319951891899109, 0.0]]
F1 for iterations:  [[0.7557834755117917, 0.0], [0.7557451441663556, 0.0], [0.7557323560428683, 0.0]]
Precision for iterations:  [[0.702111481152869, 0.0], [0.7281547787506915, 0.0], [0.7281333177519175, 0.0]]
Recall for iterations:  [[0.831995163990328, 0.0], [0.831995163990328, 0.0], [0.831995163990328, 0.0]]


#### Dataset 3B 

In [45]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part3.csv')

In [46]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [47]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID301.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [48]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [49]:
global_model = build_model((24,1,1))

In [50]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [51]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id301.hdf5')

Epoch 1/5
15/15 [==============================] - 1s 52ms/step - loss: 1.1526 - accuracy: 0.5712 - val_loss: 1.3945 - val_accuracy: 0.0366
Epoch 2/5
15/15 [==============================] - 1s 40ms/step - loss: 0.6605 - accuracy: 0.9164 - val_loss: 1.4258 - val_accuracy: 5.4975e-04
Epoch 3/5
15/15 [==============================] - 1s 40ms/step - loss: 0.3799 - accuracy: 0.9474 - val_loss: 1.4602 - val_accuracy: 0.0000e+00
Epoch 4/5
15/15 [==============================] - 1s 42ms/step - loss: 0.2627 - accuracy: 0.9562 - val_loss: 1.4928 - val_accuracy: 0.0000e+00
Epoch 5/5
15/15 [==============================] - 1s 42ms/step - loss: 0.2192 - accuracy: 0.9614 - val_loss: 1.5218 - val_accuracy: 0.0000e+00
Epoch 1/5
15/15 [==============================] - 1s 65ms/step - loss: 0.1929 - accuracy: 0.9630 - val_loss: 1.5858 - val_accuracy: 0.0000e+00
Epoch 2/5
15/15 [==============================] - 1s 43ms/step - loss: 0.1798 - accuracy: 0.9628 - val_loss: 1.6270 - val_accuracy: 8.2463e

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 7.8574e-04


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
15/15 [==============================] - 1s 53ms/step - loss: 0.1686 - accuracy: 0.9613 - val_loss: 1.7795 - val_accuracy: 5.4975e-04
Epoch 2/5
15/15 [==============================] - 1s 49ms/step - loss: 0.1645 - accuracy: 0.9616 - val_loss: 1.8444 - val_accuracy: 5.4975e-04
Epoch 3/5
15/15 [==============================] - 1s 43ms/step - loss: 0.1613 - accuracy: 0.9619 - val_loss: 1.9172 - val_accuracy: 5.4975e-04
Epoch 4/5
15/15 [==============================] - 1s 43ms/step - loss: 0.1588 - accuracy: 0.9623 - val_loss: 1.9922 - val_accuracy: 5.4975e-04
Epoch 5/5
15/15 [==============================] - 1s 48ms/step - loss: 0.1565 - accuracy: 0.9627 - val_loss: 2.0779 - val_accuracy: 2.7488e-04
Epoch 1/5
15/15 [==============================] - 1s 50ms/step - loss: 0.1515 - accuracy: 0.9643 - val_loss: 2.1660 - val_accuracy: 0.0011
Epoch 2/5
15/15 [==============================] - 1s 41ms/step - loss: 0.1493 - accuracy: 0.9649 - val_loss: 2.2868 - val_accuracy: 0.0022


...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 87/756 [==>...........................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 5.3761e-04


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [52]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.1887272596359253, 0.0], [1.0256564617156982, 0.0], [0.9147417545318604, 0.0]]
Accuracy for iterations:  [[0.8463636636734009, 0.000785740849096328], [0.8466426730155945, 0.0007443861104547977], [0.8974828124046326, 0.0005376121844165027]]
F1 for iterations:  [[0.8425355362101674, 0.0015702479338842972], [0.8454203111891219, 0.0014876647795363446], [0.8832479197090152, 0.001074646606596677]]
Precision for iterations:  [[0.867401586430612, 1.0], [0.8900050594541827, 1.0], [0.9012296140781448, 1.0]]
Recall for iterations:  [[0.8463636927273854, 0.0007857408709317233], [0.8466426932853865, 0.0007443860882511063], [0.8974827949655899, 0.0005376121748480212]]


#### Dataset 3C

In [53]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3C-Part3.csv')

In [54]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [55]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID302.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [56]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [57]:
global_model = build_model((24,1,1))

In [58]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [59]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id302.hdf5')

Epoch 1/5
4/4 [==============================] - 0s 88ms/step - loss: 1.3658 - accuracy: 0.3023 - val_loss: 1.4689 - val_accuracy: 0.0000e+00
Epoch 2/5
4/4 [==============================] - 0s 31ms/step - loss: 1.2662 - accuracy: 0.4471 - val_loss: 1.4690 - val_accuracy: 0.0000e+00
Epoch 3/5
4/4 [==============================] - 0s 34ms/step - loss: 1.1787 - accuracy: 0.5252 - val_loss: 1.4692 - val_accuracy: 0.0000e+00
Epoch 4/5
4/4 [==============================] - 0s 32ms/step - loss: 1.1040 - accuracy: 0.5595 - val_loss: 1.4702 - val_accuracy: 0.0000e+00
Epoch 5/5
4/4 [==============================] - 0s 29ms/step - loss: 1.0392 - accuracy: 0.5882 - val_loss: 1.4719 - val_accuracy: 0.0000e+00
Epoch 1/5
4/4 [==============================] - 1s 161ms/step - loss: 0.9810 - accuracy: 0.6174 - val_loss: 1.4747 - val_accuracy: 0.0000e+00
Epoch 2/5
4/4 [==============================] - 0s 28ms/step - loss: 0.9308 - accuracy: 0.6269 - val_loss: 1.4790 - val_accuracy: 0.0000e+00
Epoch

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 73/756 [=>............................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.9157

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.5039


...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 81/756 [==>...........................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.9475

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.5330


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [60]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.3564409017562866, 0.0], [1.339467167854309, 0.0], [1.3219157457351685, 0.0]]
Accuracy for iterations:  [[0.41292083263397217, 0.5038666725158691], [0.4031093120574951, 0.5662710666656494], [0.6546903252601624, 0.5330218076705933]]
F1 for iterations:  [[0.5323212407347695, 0.6700948714423209], [0.5235249961400299, 0.723081797539209], [0.7453501074322023, 0.6953871054761263]]
Precision for iterations:  [[0.8954696137612571, 1.0], [0.8960496716722028, 1.0], [0.8963425281316993, 1.0]]
Recall for iterations:  [[0.4129208258416517, 0.5038666721806377], [0.40310930621861246, 0.5662710392456888], [0.6546903093806188, 0.5330217939704727]]


#### Dataset 3D

In [61]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3D-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3D-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3D-Part3.csv')


In [62]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [63]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID303.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [64]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [65]:
global_model = build_model((24,1,1))

In [66]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [67]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id303.hdf5')

Epoch 1/5
34/34 [==============================] - 2s 49ms/step - loss: 1.0101 - accuracy: 0.6382 - val_loss: 1.2839 - val_accuracy: 0.9730
Epoch 2/5
34/34 [==============================] - 2s 45ms/step - loss: 0.2697 - accuracy: 0.9800 - val_loss: 1.1668 - val_accuracy: 0.9770
Epoch 3/5
34/34 [==============================] - 2s 48ms/step - loss: 0.1138 - accuracy: 0.9826 - val_loss: 1.0746 - val_accuracy: 0.9818
Epoch 4/5
34/34 [==============================] - 2s 46ms/step - loss: 0.0883 - accuracy: 0.9834 - val_loss: 0.9914 - val_accuracy: 0.9818
Epoch 5/5
34/34 [==============================] - 2s 52ms/step - loss: 0.0817 - accuracy: 0.9834 - val_loss: 0.9106 - val_accuracy: 0.9818
Epoch 1/5
5/5 [==============================] - 0s 93ms/step - loss: 2.1256 - accuracy: 0.6158 - val_loss: 1.5881 - val_accuracy: 0.0000e+00
Epoch 2/5
5/5 [==============================] - 0s 75ms/step - loss: 1.8321 - accuracy: 0.6158 - val_loss: 1.5287 - val_accuracy: 0.0000e+00
Epoch 3/5
5/5 [=

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


756/756 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0017


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
34/34 [==============================] - 3s 76ms/step - loss: 0.0919 - accuracy: 0.9834 - val_loss: 0.7690 - val_accuracy: 0.9819
Epoch 2/5
34/34 [==============================] - 2s 49ms/step - loss: 0.0738 - accuracy: 0.9834 - val_loss: 0.6583 - val_accuracy: 0.9829
Epoch 3/5
34/34 [==============================] - 2s 45ms/step - loss: 0.0702 - accuracy: 0.9837 - val_loss: 0.5547 - val_accuracy: 0.9835
Epoch 4/5
34/34 [==============================] - 2s 44ms/step - loss: 0.0678 - accuracy: 0.9840 - val_loss: 0.4548 - val_accuracy: 0.9842
Epoch 5/5
34/34 [==============================] - 2s 54ms/step - loss: 0.0657 - accuracy: 0.9840 - val_loss: 0.3561 - val_accuracy: 0.9846
Epoch 1/5
5/5 [==============================] - 0s 71ms/step - loss: 2.6893 - accuracy: 0.6158 - val_loss: 2.5477 - val_accuracy: 0.0000e+00
Epoch 2/5
5/5 [==============================] - 0s 39ms/step - loss: 2.3461 - accuracy: 0.6158 - val_loss: 2.3014 - val_accuracy: 0.0000e+00
Epoch 3/5
5/5 [=

...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 32/756 [>.............................] - ETA: 1s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [68]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.1719725131988525, 0.0], [0.800023078918457, 0.0], [0.47051626443862915, 0.0]]
Accuracy for iterations:  [[0.9245458245277405, 0.0016541912918910384], [0.9244063496589661, 8.270956459455192e-05], [0.9238948225975037, 0.0]]
F1 for iterations:  [[0.9003306100193208, 0.00330291895462615], [0.9004459250262938, 0.0001654054501095811], [0.9012541999069188, 0.0]]
Precision for iterations:  [[0.911393225868665, 1.0], [0.8842761197961406, 1.0], [0.8876735236727731, 0.0]]
Recall for iterations:  [[0.9245458490916982, 0.0016541913072246805], [0.9244063488126977, 8.270956536123402e-05], [0.9238948477896956, 0.0]]


#### DATASET 3E

In [69]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3E-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3E-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3E-Part3.csv')

In [70]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [71]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID304.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [72]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [73]:
global_model = build_model((24,1,1))

In [74]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [75]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id304.hdf5')

Epoch 1/5
126/126 [==============================] - 7s 54ms/step - loss: 0.4138 - accuracy: 0.9215 - val_loss: 0.9119 - val_accuracy: 0.9997
Epoch 2/5
126/126 [==============================] - 6s 49ms/step - loss: 0.0161 - accuracy: 0.9999 - val_loss: 0.4875 - val_accuracy: 1.0000
Epoch 3/5
126/126 [==============================] - 6s 51ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.1639 - val_accuracy: 1.0000
Epoch 4/5
126/126 [==============================] - 6s 51ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0319 - val_accuracy: 1.0000
Epoch 5/5
126/126 [==============================] - 7s 55ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0057 - val_accuracy: 1.0000
Epoch 1/5
17/17 [==============================] - 1s 58ms/step - loss: 5.7038 - accuracy: 0.0023 - val_loss: 2.7096 - val_accuracy: 0.0267
Epoch 2/5
17/17 [==============================] - 1s 58ms/step - loss: 3.4110 - accuracy: 0.0313 - val_loss: 1.5475 - val_accuracy: 0.1235
Epoch 3/5


c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0126


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
126/126 [==============================] - 6s 51ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 5.3238e-04 - val_accuracy: 1.0000
Epoch 2/5
126/126 [==============================] - 6s 51ms/step - loss: 3.2663e-04 - accuracy: 1.0000 - val_loss: 1.5189e-04 - val_accuracy: 1.0000
Epoch 3/5
126/126 [==============================] - 7s 52ms/step - loss: 1.3281e-04 - accuracy: 1.0000 - val_loss: 7.1471e-05 - val_accuracy: 1.0000
Epoch 4/5
126/126 [==============================] - 7s 52ms/step - loss: 5.9380e-05 - accuracy: 1.0000 - val_loss: 4.5461e-05 - val_accuracy: 1.0000
Epoch 5/5
126/126 [==============================] - 7s 53ms/step - loss: 3.9281e-05 - accuracy: 1.0000 - val_loss: 3.3125e-05 - val_accuracy: 1.0000
Epoch 1/5
17/17 [==============================] - 1s 68ms/step - loss: 4.4262 - accuracy: 0.0016 - val_loss: 1.6588 - val_accuracy: 0.0730
Epoch 2/5
17/17 [==============================] - 1s 51ms/step - loss: 1.6503 - accuracy: 0.2012 - val_loss: 0.52

...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 75/756 [=>............................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [76]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.5628840923309326, 0.0], [0.40617313981056213, 0.0], [0.41553929448127747, 0.0]]
Accuracy for iterations:  [[0.8664982318878174, 0.012571853585541248], [0.8442401885986328, 0.0011165791656821966], [0.8337466716766357, 0.0]]
F1 for iterations:  [[0.8487789989915046, 0.02483152950786196], [0.7865356967049646, 0.0022306675479180434], [0.7603482021009152, 0.0]]
Precision for iterations:  [[0.8604512968539972, 1.0], [0.806134363984871, 1.0], [0.7813869474090002, 0.0]]
Recall for iterations:  [[0.866498232996466, 0.012571853934907572], [0.844240188480377, 0.0011165791323766593], [0.833746667493335, 0.0]]


### Experimenting with epochs and iterations 

Use of dataset 3C, considering it contains an equal partition of the data between the 3 nodes. 

#### #Epochs = 10, #Iterations = 10

In [84]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part3.csv')


In [85]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [86]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID310.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [87]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [88]:
global_model = build_model((24,1,1))

In [89]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [90]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id310.hdf5')

Epoch 1/10
15/15 [==============================] - 1s 64ms/step - loss: 1.4004 - accuracy: 0.2339 - val_loss: 1.3880 - val_accuracy: 0.0132
Epoch 2/10
15/15 [==============================] - 1s 55ms/step - loss: 0.7849 - accuracy: 0.7902 - val_loss: 1.4149 - val_accuracy: 0.0099
Epoch 3/10
15/15 [==============================] - 1s 61ms/step - loss: 0.4630 - accuracy: 0.9495 - val_loss: 1.4448 - val_accuracy: 0.0060
Epoch 4/10
15/15 [==============================] - 1s 56ms/step - loss: 0.3126 - accuracy: 0.9530 - val_loss: 1.4752 - val_accuracy: 0.0019
Epoch 5/10
15/15 [==============================] - 1s 55ms/step - loss: 0.2459 - accuracy: 0.9552 - val_loss: 1.5046 - val_accuracy: 0.0011
Epoch 6/10
15/15 [==============================] - 1s 54ms/step - loss: 0.2163 - accuracy: 0.9576 - val_loss: 1.5321 - val_accuracy: 0.0011
Epoch 7/10
15/15 [==============================] - 1s 57ms/step - loss: 0.2025 - accuracy: 0.9600 - val_loss: 1.5591 - val_accuracy: 0.0011
Epoch 8/10
15

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


756/756 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0026


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

Epoch 1/10
15/15 [==============================] - 1s 70ms/step - loss: 0.1575 - accuracy: 0.9625 - val_loss: 2.1861 - val_accuracy: 0.0014
Epoch 2/10
15/15 [==============================] - 1s 50ms/step - loss: 0.1545 - accuracy: 0.9630 - val_loss: 2.3001 - val_accuracy: 0.0014
Epoch 3/10
15/15 [==============================] - 1s 54ms/step - loss: 0.1523 - accuracy: 0.9638 - val_loss: 2.4124 - val_accuracy: 0.0014
Epoch 4/10
15/15 [==============================] - 1s 56ms/step - loss: 0.1509 - accuracy: 0.9642 - val_loss: 2.5315 - val_accuracy: 0.0014
Epoch 5/10
15/15 [==============================] - 1s 56ms/step - loss: 0.1502 - accuracy: 0.9646 - val_loss: 2.6696 - val_accuracy: 0.0014
Epoch 6/10
15/15 [==============================] - 1s 52ms/step - loss: 0.1487 - accuracy: 0.9652 - val_loss: 2.7996 - val_accuracy: 0.0014
Epoch 7/10
15/15 [==============================] - 1s 57ms/step - loss: 0.1478 - accuracy: 0.9652 - val_loss: 2.9563 - val_accuracy: 0.0014
Epoch 8/10
15

...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 94/756 [==>...........................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0020    

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0319


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [91]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.0237730741500854, 0.0], [0.8249384164810181, 0.0], [0.9147790670394897, 0.0], [1.074325442314148, 0.0], [1.217869520187378, 0.0], [1.3567068576812744, 0.0], [1.484483003616333, 0.0], [1.5786012411117554, 0.0], [1.6685655117034912, 0.0], [1.7719646692276, 0.0]]
Accuracy for iterations:  [[0.9236623644828796, 0.0026467060670256615], [0.9085963368415833, 0.003308382583782077], [0.8968937993049622, 0.0014060626272112131], [0.82808917760849, 0.0014887722209095955], [0.7499535083770752, 0.0025226417928934097], [0.548003613948822, 0.001985029550269246], [0.5481120944023132, 0.0030188991222530603], [0.5173445343971252, 0.010007857345044613], [0.3387066721916199, 0.01670733280479908], [0.21197842061519623, 0.031884536147117615]]
F1 for iterations:  [[0.902279788651716, 0.0052794390595999165], [0.8926317561239985, 0.006594946622150777], [0.8832665010353082, 0.0028081767499483793], [0.8322452060127586, 0.002973118057562869], [0.7749116402238502, 0.0050325880702912305], [

#### #Epochs = 5, #Iterations = 5

In [92]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part3.csv')


In [93]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [94]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID311.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [95]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [96]:
global_model = build_model((24,1,1))

In [97]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [98]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id311.hdf5')

Epoch 1/5
15/15 [==============================] - 1s 51ms/step - loss: 0.7485 - accuracy: 0.6808 - val_loss: 1.4227 - val_accuracy: 0.0000e+00
Epoch 2/5
15/15 [==============================] - 1s 40ms/step - loss: 0.4078 - accuracy: 0.9541 - val_loss: 1.4464 - val_accuracy: 0.0000e+00
Epoch 3/5
15/15 [==============================] - 1s 41ms/step - loss: 0.2669 - accuracy: 0.9566 - val_loss: 1.4729 - val_accuracy: 0.0000e+00
Epoch 4/5
15/15 [==============================] - 1s 41ms/step - loss: 0.2165 - accuracy: 0.9579 - val_loss: 1.4985 - val_accuracy: 0.0000e+00
Epoch 5/5
15/15 [==============================] - 1s 41ms/step - loss: 0.1986 - accuracy: 0.9602 - val_loss: 1.5230 - val_accuracy: 0.0000e+00
Epoch 1/5
15/15 [==============================] - 1s 51ms/step - loss: 0.1834 - accuracy: 0.9626 - val_loss: 1.5605 - val_accuracy: 2.7488e-04
Epoch 2/5
15/15 [==============================] - 1s 45ms/step - loss: 0.1710 - accuracy: 0.9628 - val_loss: 1.5963 - val_accuracy: 0.0

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 88/756 [==>...........................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 4.1355e-05


...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 66/756 [=>............................] - ETA: 1s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 8.6845e-04


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [99]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.2214161157608032, 0.0], [1.0118076801300049, 0.0], [0.8539761304855347, 0.0], [0.8654507994651794, 0.0], [0.9344009757041931, 0.0]]
Accuracy for iterations:  [[0.8110701441764832, 4.135478229727596e-05], [0.9233988523483276, 4.135478229727596e-05], [0.9228873252868652, 8.270956459455192e-05], [0.8311271667480469, 0.0005376121844165027], [0.8063116073608398, 0.0008684504427947104]]
F1 for iterations:  [[0.8192511653591829, 8.270614506657845e-05], [0.9027045462324034, 8.270614506657845e-05], [0.9021710101400904, 0.0001654054501095811], [0.8330765817023695, 0.001074646606596677], [0.8122254879306584, 0.0017353937691099908]]
Precision for iterations:  [[0.8690046798810036, 1.0], [0.9073787378424245, 1.0], [0.9063403946297581, 1.0], [0.8794303545171928, 1.0], [0.8568507618475205, 1.0]]
Recall for iterations:  [[0.8110701221402443, 4.135478268061701e-05], [0.9233988467976936, 4.135478268061701e-05], [0.9228873457746916, 8.270956536123402e-05], [0.8311271622543245, 0

#### #Epochs = 5, #Iterations = 10

In [100]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part3.csv')


In [101]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [102]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID312.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [103]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [104]:
global_model = build_model((24,1,1))

In [105]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [106]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id312.hdf5')

Epoch 1/5
15/15 [==============================] - 1s 52ms/step - loss: 1.1180 - accuracy: 0.6776 - val_loss: 1.4099 - val_accuracy: 0.0679
Epoch 2/5
15/15 [==============================] - 1s 40ms/step - loss: 0.6454 - accuracy: 0.9178 - val_loss: 1.4348 - val_accuracy: 0.0047
Epoch 3/5
15/15 [==============================] - 1s 41ms/step - loss: 0.3934 - accuracy: 0.9509 - val_loss: 1.4622 - val_accuracy: 0.0022
Epoch 4/5
15/15 [==============================] - 1s 41ms/step - loss: 0.2780 - accuracy: 0.9544 - val_loss: 1.4915 - val_accuracy: 0.0014
Epoch 5/5
15/15 [==============================] - 1s 41ms/step - loss: 0.2301 - accuracy: 0.9546 - val_loss: 1.5212 - val_accuracy: 0.0014
Epoch 1/5
15/15 [==============================] - 1s 63ms/step - loss: 0.1960 - accuracy: 0.9574 - val_loss: 1.5872 - val_accuracy: 5.4975e-04
Epoch 2/5
15/15 [==============================] - 1s 40ms/step - loss: 0.1784 - accuracy: 0.9594 - val_loss: 1.6412 - val_accuracy: 8.2463e-04
Epoch 3/5
15

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0095


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
15/15 [==============================] - 1s 67ms/step - loss: 0.1669 - accuracy: 0.9622 - val_loss: 1.8009 - val_accuracy: 0.0000e+00
Epoch 2/5
15/15 [==============================] - 1s 43ms/step - loss: 0.1617 - accuracy: 0.9624 - val_loss: 1.8758 - val_accuracy: 0.0000e+00
Epoch 3/5
15/15 [==============================] - 1s 42ms/step - loss: 0.1588 - accuracy: 0.9625 - val_loss: 1.9430 - val_accuracy: 0.0000e+00
Epoch 4/5
15/15 [==============================] - 1s 42ms/step - loss: 0.1572 - accuracy: 0.9625 - val_loss: 2.0184 - val_accuracy: 0.0000e+00
Epoch 5/5
15/15 [==============================] - 1s 47ms/step - loss: 0.1550 - accuracy: 0.9627 - val_loss: 2.1028 - val_accuracy: 0.0000e+00
Epoch 1/5
15/15 [==============================] - 1s 61ms/step - loss: 0.1506 - accuracy: 0.9650 - val_loss: 2.1979 - val_accuracy: 8.2463e-04
Epoch 2/5
15/15 [==============================] - 1s 46ms/step - loss: 0.1482 - accuracy: 0.9649 - val_loss: 2.2954 - val_accuracy: 8.2

...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 55/756 [=>............................] - ETA: 1s - loss: 0.0000e+00 - accuracy: 0.0011    

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0161


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [107]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.1798760890960693, 0.0], [1.0976953506469727, 0.0], [1.0861066579818726, 0.0], [1.1553384065628052, 0.0], [1.271125316619873, 0.0], [1.382882833480835, 0.0], [1.5127978324890137, 0.0], [1.6245250701904297, 0.0], [1.7884374856948853, 0.0], [1.8808571100234985, 0.0]]
Accuracy for iterations:  [[0.9123473167419434, 0.009511600248515606], [0.5660921335220337, 0.008519085124135017], [0.547213077545166, 0.008560439571738243], [0.5720441341400146, 0.009511600248515606], [0.5031620264053345, 0.010297341272234917], [0.5569781064987183, 0.010959017090499401], [0.4714334309101105, 0.011083081364631653], [0.27986857295036316, 0.012778627686202526], [0.13485026359558105, 0.013357594609260559], [0.13483476638793945, 0.016087010502815247]]
F1 for iterations:  [[0.8914495420950246, 0.018843963786817422], [0.6268891873320014, 0.016894246934842337], [0.6099492831593067, 0.016975561751681155], [0.6321475005474692, 0.018843963786817422], [0.5688086766181324, 0.020384772820302908],

#### #Epochs = 5, #Iterations = 15

In [108]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_11572/2294361559.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')


In [109]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 15

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [110]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID313.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [111]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [112]:
global_model = build_model((24,1,1))

In [113]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [114]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id313.hdf5')

Epoch 1/5
15/15 [==============================] - 1s 56ms/step - loss: 1.3037 - accuracy: 0.3619 - val_loss: 1.4089 - val_accuracy: 0.0478
Epoch 2/5
15/15 [==============================] - 1s 51ms/step - loss: 0.7652 - accuracy: 0.8485 - val_loss: 1.4299 - val_accuracy: 8.2463e-04
Epoch 3/5
15/15 [==============================] - 1s 55ms/step - loss: 0.4578 - accuracy: 0.9477 - val_loss: 1.4532 - val_accuracy: 8.2463e-04
Epoch 4/5
15/15 [==============================] - 1s 62ms/step - loss: 0.3100 - accuracy: 0.9546 - val_loss: 1.4790 - val_accuracy: 5.4975e-04
Epoch 5/5
15/15 [==============================] - 1s 57ms/step - loss: 0.2450 - accuracy: 0.9557 - val_loss: 1.5068 - val_accuracy: 5.4975e-04
Epoch 1/5
15/15 [==============================] - 1s 66ms/step - loss: 0.2013 - accuracy: 0.9598 - val_loss: 1.5573 - val_accuracy: 0.0000e+00
Epoch 2/5
15/15 [==============================] - 1s 55ms/step - loss: 0.1783 - accuracy: 0.9625 - val_loss: 1.6069 - val_accuracy: 0.0000e

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 65/756 [=>............................] - ETA: 1s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 63/756 [=>............................] - ETA: 1s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0017


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [115]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.1897985935211182, 0.0], [1.004499912261963, 0.0], [0.8885879516601562, 0.0], [0.9194231629371643, 0.0], [1.039891242980957, 0.0], [1.1802499294281006, 0.0], [1.2594116926193237, 0.0], [1.3347879648208618, 0.0], [1.418838381767273, 0.0], [1.506758689880371, 0.0], [1.5601729154586792, 0.0], [1.6428050994873047, 0.0], [1.7160673141479492, 0.0], [1.793906331062317, 0.0], [1.8375529050827026, 0.0]]
Accuracy for iterations:  [[0.8465341925621033, 0.0], [0.8465806841850281, 4.135478229727596e-05], [0.8466581702232361, 0.0002067739114863798], [0.8467047214508057, 0.0011165791656821966], [0.8118296265602112, 0.0022331583313643932], [0.6897203922271729, 0.0024399321991950274], [0.5923646688461304, 0.0022331583313643932], [0.4451608955860138, 0.002026384463533759], [0.2608500123023987, 0.002026384463533759], [0.10774071514606476, 0.002026384463533759], [0.10760121792554855, 0.0019023199565708637], [0.10761671513319016, 0.0019023199565708637], [0.10774071514606476, 0.0019